In [275]:
#import streamlit as st
import pandas as pd
import numpy as np
from sklearn import neighbors
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import joblib
import googlemaps


In [276]:
!pwd

/Users/malou/code/MalouO/small_business/notebooks/3_Prediction


In [277]:
data = pd.read_csv('../../small_business/data/restaurants.csv')

In [278]:

def restaurant_data():

    data = pd.read_csv('../../small_business/data/restaurants.csv')
    data = data.drop(columns = 'Unnamed: 0')
    data = data.dropna()
    X = data.drop(columns=['rating','name', 'address', 'label', 'curb_pickup', 'drive_through', 'postal_code', 'no_del_exp', 'municipality', 'review_count'])
    y = data['rating']
    return X, y

def latitude(address):
    key = pd.read_csv("../raw_data/api_key_vb.csv", header=None)
    key = key.loc[0][0]
    gmaps = googlemaps.Client(key=key)
    geocode_result = gmaps.geocode(address)
    return float(geocode_result[0]['geometry']['location']['lat'])

def longitude(address):
    key = pd.read_csv("../raw_data/api_key_vb.csv", header=None)
    key = key.loc[0][0]
    gmaps = googlemaps.Client(key=key)
    geocode_result = gmaps.geocode(address)
    return float(geocode_result[0]['geometry']['location']['lng'])

def address_imputer(address, neighborhood, X):
    if address == None:
        all_latitude = pd.DataFrame(X['latitude'].groupby(X['neighborhood']).mean())
        all_longitude = pd.DataFrame(X['longitude'].groupby(X['neighborhood']).mean())
        for place in all_latitude.index:
            if place == neighborhood:
                lattitude_replace = all_latitude.loc[place].values[0]
        for place in all_longitude.index:
            if place == neighborhood:
                longitude_replace = all_longitude.loc[place].values[0]
        return lattitude_replace, longitude_replace
    else:
        longitude1 = longitude(address)
        latitude1 = latitude(address)
        return latitude1, longitude1

def get_fitted_pipe(X):
    num_transformer = StandardScaler()
    price_transformer = SimpleImputer(strategy="mean")
    cat_transformer = OneHotEncoder(handle_unknown='ignore')
    preproc = make_column_transformer((price_transformer, ['price']),
                                       (num_transformer, ['latitude', 'longitude']),
                                        (cat_transformer, ['neighborhood', 'type']), remainder='passthrough')
    pipe = make_pipeline(preproc)
    pipe.fit_transform(X)
    return pipe

def target_X(type_of_food, price, neighborhood, latitude, longitude):
    new_df=pd.DataFrame(columns=['type','price','latitude','longitude', 'dine_in', 'takeaway','delivery', 'neighborhood'])
    new_row = {'type':type_of_food, 'price': price, 'latitude':latitude,'longitude':longitude,'takeaway':1,'dine_in':1,'delivery':1,  'neighborhood': neighborhood}
    X_user= new_df.append(new_row, ignore_index=True)
    return X_user

def build_y(y):
    y_class=pd.cut(x=y, bins=[0,4.5, 5],
                        labels=["bad idea!", "good idea!"])
    return y_class

def train_model(pipeline, X, y_class):

    knn = KNeighborsClassifier(n_neighbors = 5,weights= 'uniform')
    knn.fit(pipeline.fit_transform(X), y_class)
    filename = 'knn_model.joblib'
    joblib.dump(knn, filename)


def predict(X_user_transformed):

    loaded_model = joblib.load('knn_model.joblib')
    y_probxgb = loaded_model.predict_proba(X_user_transformed)
    if y_probxgb[0][0]< 0.8:
            pred =  'bad idea!'
    else:
            pred = 'good idea!'
    return pred

def return_predicted_probability(X_user_transformed):
    loaded_model = joblib.load('knn_model.joblib')
    y_probxgb = loaded_model.predict_proba(X_user_transformed)
    return y_probxgb

def all_results(type_of_food, price, neighborhood, latitude, longitude):
        loaded_model = joblib.load('knn_model.joblib')
        y_probxgb = loaded_model.predict_proba(X_user_transformed)


def neighbours(X_user_transformed):
    loaded_model = joblib.load('knn_model.joblib')
    neighbors =  loaded_model.kneighbors(X_user_transformed, n_neighbors=10, return_distance=False)
    return neighbors

if __name__ == '__main__':
    X, y = restaurant_data()
    pipeline = get_fitted_pipe(X)
    type_of_food = 'brunch'
    neighborhood = 'Graça'
    address = None
    price = 2
    latitude1, longitude1 = address_imputer(address, neighborhood, X)
    X_user = target_X(type_of_food = type_of_food, price = price, neighborhood = neighborhood, latitude = latitude1, longitude=longitude1)
    X_user_transformed = pipeline.transform(X_user)
    y_class = build_y(y)
    train_model(pipeline, X, y_class)
    predict(X_user_transformed)
    neighbours(X_user_transformed)

def output_model(data, type_of_food, price, address, neighborhood):
    X, y = restaurant_data()
    pipeline = get_fitted_pipe(X)
    type_of_food = 'brunch'
    neighborhood = 'Graça'
    address = None
    price = 2
    latitude, longitude = address_imputer(address, neighborhood, X)
    X_user = target_X(type_of_food = type_of_food, price = price, neighborhood = neighborhood, latitude = latitude, longitude=longitude)
    X_user_transformed = pipeline.transform(X_user)
    y_class = build_y(y)
    predict(X_user_transformed)
    neighbours(X_user_transformed)


In [279]:
def get_imputed_coords(neighborhood, X):
        all_latitude = pd.DataFrame(X['latitude'].groupby(X['neighborhood']).mean())
        all_longitude = pd.DataFrame(X['longitude'].groupby(X['neighborhood']).mean())
        for place in all_latitude.index:
            if place == neighborhood:
                lattitude = all_latitude.loc[place].values[0]
        for place in all_longitude.index:
            if place in neighborhood:
                longitude = all_longitude.loc[place].values[0]
        return lattitude, longitude

In [280]:
get_imputed_coords('Graça', X)

(38.71728895789474, -9.130980542105263)

In [281]:
def transform_input(type_of_food, price, neighborhood, latitude, longitude):
            new_df=pd.DataFrame(columns=['type','price','latitude','longitude', 'dine_in', 'takeaway','delivery', 'neighborhood'])
            new_row = {'type':type_of_food, 'price': price, 'latitude':latitude,'longitude':longitude,'takeaway':1,'dine_in':1,'delivery':1,  'neighborhood': neighborhood}
            X_user= new_df.append(new_row, ignore_index=True)
            return X_user

In [282]:
all_scenarios = []
    for n in neighborhood: 
        for p in price: 
            for t in type_of_food:

                all_scenarios.append({{type_of_food:t, neighborhood:n, price:p }: y_probxgb})

    df = pd.Dataframe(all_scenarios)
    df.to_csv('all_scores.csv')
                

IndentationError: unexpected indent (198569215.py, line 2)

In [283]:




loaded_model = joblib.load('knn_model.joblib')
y_probxgb = loaded_model.predict_proba(transformed_input)

NameError: name 'transformed_input' is not defined

In [284]:
X, y = restaurant_data()
type_of_food = list(X['type'].unique())
price = X['price'].unique()
neighborhood = X['neighborhood'].unique()
for t in type_of_food:
    for p in price:
        for n in neighborhood:
            print(t, p, n)


brunch 2 Graça
brunch 2 Prazeres
brunch 2 Santa Engrácia
brunch 2 São Paulo
brunch 2 Lapa
brunch 2 Santa Catarina
brunch 2 Alto do Pina
brunch 2 Encarnação
brunch 2 Santa Isabel
brunch 2 Socorro
brunch 2 Anjos
brunch 2 Santo Estevão
brunch 2 Pena
brunch 2 Madalena
brunch 2 Santos-o-Velho
brunch 2 São Vicente de Fora
brunch 2 Mercês
brunch 2 Coração de Jesus
brunch 2 São José
brunch 2 Sacramento
brunch 2 São Miguel
brunch 2 Mártires
brunch 2 São Cristóvão
brunch 2 São Mamede
brunch 2 Sé
brunch 2 São Nicolau
brunch 2 Campolide
brunch 2 Santa Justa
brunch 2 São Jorge de Arroios
brunch 2 Santiago
brunch 2 Santo Condestável
brunch 2 Castelo
brunch 2 Alvalade
brunch 2 São Domingos de Benfica
brunch 2 Nossa Senhora de Fátima
brunch 2 Almargem do Bispo
brunch 2 São Sebastião da Pedreira
brunch 2 Santa Maria dos Olivais
brunch 1 Graça
brunch 1 Prazeres
brunch 1 Santa Engrácia
brunch 1 São Paulo
brunch 1 Lapa
brunch 1 Santa Catarina
brunch 1 Alto do Pina
brunch 1 Encarnação
brunch 1 Santa Isabel

european 3 Santos-o-Velho
european 3 São Vicente de Fora
european 3 Mercês
european 3 Coração de Jesus
european 3 São José
european 3 Sacramento
european 3 São Miguel
european 3 Mártires
european 3 São Cristóvão
european 3 São Mamede
european 3 Sé
european 3 São Nicolau
european 3 Campolide
european 3 Santa Justa
european 3 São Jorge de Arroios
european 3 Santiago
european 3 Santo Condestável
european 3 Castelo
european 3 Alvalade
european 3 São Domingos de Benfica
european 3 Nossa Senhora de Fátima
european 3 Almargem do Bispo
european 3 São Sebastião da Pedreira
european 3 Santa Maria dos Olivais
chicken 2 Graça
chicken 2 Prazeres
chicken 2 Santa Engrácia
chicken 2 São Paulo
chicken 2 Lapa
chicken 2 Santa Catarina
chicken 2 Alto do Pina
chicken 2 Encarnação
chicken 2 Santa Isabel
chicken 2 Socorro
chicken 2 Anjos
chicken 2 Santo Estevão
chicken 2 Pena
chicken 2 Madalena
chicken 2 Santos-o-Velho
chicken 2 São Vicente de Fora
chicken 2 Mercês
chicken 2 Coração de Jesus
chicken 2 São Jo

south_america 2 São Miguel
south_america 2 Mártires
south_america 2 São Cristóvão
south_america 2 São Mamede
south_america 2 Sé
south_america 2 São Nicolau
south_america 2 Campolide
south_america 2 Santa Justa
south_america 2 São Jorge de Arroios
south_america 2 Santiago
south_america 2 Santo Condestável
south_america 2 Castelo
south_america 2 Alvalade
south_america 2 São Domingos de Benfica
south_america 2 Nossa Senhora de Fátima
south_america 2 Almargem do Bispo
south_america 2 São Sebastião da Pedreira
south_america 2 Santa Maria dos Olivais
south_america 1 Graça
south_america 1 Prazeres
south_america 1 Santa Engrácia
south_america 1 São Paulo
south_america 1 Lapa
south_america 1 Santa Catarina
south_america 1 Alto do Pina
south_america 1 Encarnação
south_america 1 Santa Isabel
south_america 1 Socorro
south_america 1 Anjos
south_america 1 Santo Estevão
south_america 1 Pena
south_america 1 Madalena
south_america 1 Santos-o-Velho
south_america 1 São Vicente de Fora
south_america 1 Mer

grill 1 Campolide
grill 1 Santa Justa
grill 1 São Jorge de Arroios
grill 1 Santiago
grill 1 Santo Condestável
grill 1 Castelo
grill 1 Alvalade
grill 1 São Domingos de Benfica
grill 1 Nossa Senhora de Fátima
grill 1 Almargem do Bispo
grill 1 São Sebastião da Pedreira
grill 1 Santa Maria dos Olivais
grill 4 Graça
grill 4 Prazeres
grill 4 Santa Engrácia
grill 4 São Paulo
grill 4 Lapa
grill 4 Santa Catarina
grill 4 Alto do Pina
grill 4 Encarnação
grill 4 Santa Isabel
grill 4 Socorro
grill 4 Anjos
grill 4 Santo Estevão
grill 4 Pena
grill 4 Madalena
grill 4 Santos-o-Velho
grill 4 São Vicente de Fora
grill 4 Mercês
grill 4 Coração de Jesus
grill 4 São José
grill 4 Sacramento
grill 4 São Miguel
grill 4 Mártires
grill 4 São Cristóvão
grill 4 São Mamede
grill 4 Sé
grill 4 São Nicolau
grill 4 Campolide
grill 4 Santa Justa
grill 4 São Jorge de Arroios
grill 4 Santiago
grill 4 Santo Condestável
grill 4 Castelo
grill 4 Alvalade
grill 4 São Domingos de Benfica
grill 4 Nossa Senhora de Fátima
grill 4 

pizza 4 Socorro
pizza 4 Anjos
pizza 4 Santo Estevão
pizza 4 Pena
pizza 4 Madalena
pizza 4 Santos-o-Velho
pizza 4 São Vicente de Fora
pizza 4 Mercês
pizza 4 Coração de Jesus
pizza 4 São José
pizza 4 Sacramento
pizza 4 São Miguel
pizza 4 Mártires
pizza 4 São Cristóvão
pizza 4 São Mamede
pizza 4 Sé
pizza 4 São Nicolau
pizza 4 Campolide
pizza 4 Santa Justa
pizza 4 São Jorge de Arroios
pizza 4 Santiago
pizza 4 Santo Condestável
pizza 4 Castelo
pizza 4 Alvalade
pizza 4 São Domingos de Benfica
pizza 4 Nossa Senhora de Fátima
pizza 4 Almargem do Bispo
pizza 4 São Sebastião da Pedreira
pizza 4 Santa Maria dos Olivais
pizza 3 Graça
pizza 3 Prazeres
pizza 3 Santa Engrácia
pizza 3 São Paulo
pizza 3 Lapa
pizza 3 Santa Catarina
pizza 3 Alto do Pina
pizza 3 Encarnação
pizza 3 Santa Isabel
pizza 3 Socorro
pizza 3 Anjos
pizza 3 Santo Estevão
pizza 3 Pena
pizza 3 Madalena
pizza 3 Santos-o-Velho
pizza 3 São Vicente de Fora
pizza 3 Mercês
pizza 3 Coração de Jesus
pizza 3 São José
pizza 3 Sacramento
pizza 

burger 2 Santos-o-Velho
burger 2 São Vicente de Fora
burger 2 Mercês
burger 2 Coração de Jesus
burger 2 São José
burger 2 Sacramento
burger 2 São Miguel
burger 2 Mártires
burger 2 São Cristóvão
burger 2 São Mamede
burger 2 Sé
burger 2 São Nicolau
burger 2 Campolide
burger 2 Santa Justa
burger 2 São Jorge de Arroios
burger 2 Santiago
burger 2 Santo Condestável
burger 2 Castelo
burger 2 Alvalade
burger 2 São Domingos de Benfica
burger 2 Nossa Senhora de Fátima
burger 2 Almargem do Bispo
burger 2 São Sebastião da Pedreira
burger 2 Santa Maria dos Olivais
burger 1 Graça
burger 1 Prazeres
burger 1 Santa Engrácia
burger 1 São Paulo
burger 1 Lapa
burger 1 Santa Catarina
burger 1 Alto do Pina
burger 1 Encarnação
burger 1 Santa Isabel
burger 1 Socorro
burger 1 Anjos
burger 1 Santo Estevão
burger 1 Pena
burger 1 Madalena
burger 1 Santos-o-Velho
burger 1 São Vicente de Fora
burger 1 Mercês
burger 1 Coração de Jesus
burger 1 São José
burger 1 Sacramento
burger 1 São Miguel
burger 1 Mártires
burger

pastry 1 São Miguel
pastry 1 Mártires
pastry 1 São Cristóvão
pastry 1 São Mamede
pastry 1 Sé
pastry 1 São Nicolau
pastry 1 Campolide
pastry 1 Santa Justa
pastry 1 São Jorge de Arroios
pastry 1 Santiago
pastry 1 Santo Condestável
pastry 1 Castelo
pastry 1 Alvalade
pastry 1 São Domingos de Benfica
pastry 1 Nossa Senhora de Fátima
pastry 1 Almargem do Bispo
pastry 1 São Sebastião da Pedreira
pastry 1 Santa Maria dos Olivais
pastry 4 Graça
pastry 4 Prazeres
pastry 4 Santa Engrácia
pastry 4 São Paulo
pastry 4 Lapa
pastry 4 Santa Catarina
pastry 4 Alto do Pina
pastry 4 Encarnação
pastry 4 Santa Isabel
pastry 4 Socorro
pastry 4 Anjos
pastry 4 Santo Estevão
pastry 4 Pena
pastry 4 Madalena
pastry 4 Santos-o-Velho
pastry 4 São Vicente de Fora
pastry 4 Mercês
pastry 4 Coração de Jesus
pastry 4 São José
pastry 4 Sacramento
pastry 4 São Miguel
pastry 4 Mártires
pastry 4 São Cristóvão
pastry 4 São Mamede
pastry 4 Sé
pastry 4 São Nicolau
pastry 4 Campolide
pastry 4 Santa Justa
pastry 4 São Jorge de A

In [285]:
X, y = restaurant_data()


def all_scores():
    X, y = restaurant_data()
    all_scenarios = []
    address = None
    type_of_food = list(X['type'].unique())
    neighborhood = list(X['neighborhood'].unique())
    price = list(X['price'].unique())
    
    pipeline = get_fitted_pipe(X)
    for n in neighborhood:  
        for t in type_of_food:
            for p in price:
                latitude1, longitude1 = get_imputed_coords(address)
                X_user = transform_input(t, p, n, latitude, longitude)
                X_transformed = pipeline.transform(X_user)
                loaded_model = joblib.load('knn_model.joblib')
                y_predicted_prob = loaded_model.predict_proba(X_user)
                all_scenarios.append({{type_of_food:t, neighborhood:n, price:p}: y_probxgb})
    df = pd.Dataframe(all_scenarios)
    df.to_csv('all_scores.csv')
                
                
                
                


In [286]:
def get_lat():
    neighborhood = X['neighborhood'].unique()
    for n in neighborhood:
        latitude1, longitude1 = get_imputed_coords(n, X)
        for t in type_of_food:
            for p in price:
                print(latitude1, t, p, n)
        

In [287]:
get_lat()

38.71728895789474 brunch 2 Graça
38.71728895789474 brunch 1 Graça
38.71728895789474 brunch 4 Graça
38.71728895789474 brunch 3 Graça
38.71728895789474 cafe 2 Graça
38.71728895789474 cafe 1 Graça
38.71728895789474 cafe 4 Graça
38.71728895789474 cafe 3 Graça
38.71728895789474 mediterranean 2 Graça
38.71728895789474 mediterranean 1 Graça
38.71728895789474 mediterranean 4 Graça
38.71728895789474 mediterranean 3 Graça
38.71728895789474 european 2 Graça
38.71728895789474 european 1 Graça
38.71728895789474 european 4 Graça
38.71728895789474 european 3 Graça
38.71728895789474 chicken 2 Graça
38.71728895789474 chicken 1 Graça
38.71728895789474 chicken 4 Graça
38.71728895789474 chicken 3 Graça
38.71728895789474 bar 2 Graça
38.71728895789474 bar 1 Graça
38.71728895789474 bar 4 Graça
38.71728895789474 bar 3 Graça
38.71728895789474 seafood 2 Graça
38.71728895789474 seafood 1 Graça
38.71728895789474 seafood 4 Graça
38.71728895789474 seafood 3 Graça
38.71728895789474 south_america 2 Graça
38.717288957

38.710518906249995 cafe 2 Santa Catarina
38.710518906249995 cafe 1 Santa Catarina
38.710518906249995 cafe 4 Santa Catarina
38.710518906249995 cafe 3 Santa Catarina
38.710518906249995 mediterranean 2 Santa Catarina
38.710518906249995 mediterranean 1 Santa Catarina
38.710518906249995 mediterranean 4 Santa Catarina
38.710518906249995 mediterranean 3 Santa Catarina
38.710518906249995 european 2 Santa Catarina
38.710518906249995 european 1 Santa Catarina
38.710518906249995 european 4 Santa Catarina
38.710518906249995 european 3 Santa Catarina
38.710518906249995 chicken 2 Santa Catarina
38.710518906249995 chicken 1 Santa Catarina
38.710518906249995 chicken 4 Santa Catarina
38.710518906249995 chicken 3 Santa Catarina
38.710518906249995 bar 2 Santa Catarina
38.710518906249995 bar 1 Santa Catarina
38.710518906249995 bar 4 Santa Catarina
38.710518906249995 bar 3 Santa Catarina
38.710518906249995 seafood 2 Santa Catarina
38.710518906249995 seafood 1 Santa Catarina
38.710518906249995 seafood 4 San

38.720276356 pizza 1 Santa Isabel
38.720276356 pizza 4 Santa Isabel
38.720276356 pizza 3 Santa Isabel
38.720276356 japanese 2 Santa Isabel
38.720276356 japanese 1 Santa Isabel
38.720276356 japanese 4 Santa Isabel
38.720276356 japanese 3 Santa Isabel
38.720276356 pasta 2 Santa Isabel
38.720276356 pasta 1 Santa Isabel
38.720276356 pasta 4 Santa Isabel
38.720276356 pasta 3 Santa Isabel
38.720276356 mexican 2 Santa Isabel
38.720276356 mexican 1 Santa Isabel
38.720276356 mexican 4 Santa Isabel
38.720276356 mexican 3 Santa Isabel
38.720276356 africa_me 2 Santa Isabel
38.720276356 africa_me 1 Santa Isabel
38.720276356 africa_me 4 Santa Isabel
38.720276356 africa_me 3 Santa Isabel
38.720276356 burger 2 Santa Isabel
38.720276356 burger 1 Santa Isabel
38.720276356 burger 4 Santa Isabel
38.720276356 burger 3 Santa Isabel
38.720276356 italian 2 Santa Isabel
38.720276356 italian 1 Santa Isabel
38.720276356 italian 4 Santa Isabel
38.720276356 italian 3 Santa Isabel
38.720276356 asian 2 Santa Isabel


38.718407094999996 bistro 1 Pena
38.718407094999996 bistro 4 Pena
38.718407094999996 bistro 3 Pena
38.718407094999996 grill 2 Pena
38.718407094999996 grill 1 Pena
38.718407094999996 grill 4 Pena
38.718407094999996 grill 3 Pena
38.718407094999996 indian_nepalese 2 Pena
38.718407094999996 indian_nepalese 1 Pena
38.718407094999996 indian_nepalese 4 Pena
38.718407094999996 indian_nepalese 3 Pena
38.718407094999996 portuguese 2 Pena
38.718407094999996 portuguese 1 Pena
38.718407094999996 portuguese 4 Pena
38.718407094999996 portuguese 3 Pena
38.718407094999996 pizza 2 Pena
38.718407094999996 pizza 1 Pena
38.718407094999996 pizza 4 Pena
38.718407094999996 pizza 3 Pena
38.718407094999996 japanese 2 Pena
38.718407094999996 japanese 1 Pena
38.718407094999996 japanese 4 Pena
38.718407094999996 japanese 3 Pena
38.718407094999996 pasta 2 Pena
38.718407094999996 pasta 1 Pena
38.718407094999996 pasta 4 Pena
38.718407094999996 pasta 3 Pena
38.718407094999996 mexican 2 Pena
38.718407094999996 mexican 

38.71461291666666 cafe 4 São Vicente de Fora
38.71461291666666 cafe 3 São Vicente de Fora
38.71461291666666 mediterranean 2 São Vicente de Fora
38.71461291666666 mediterranean 1 São Vicente de Fora
38.71461291666666 mediterranean 4 São Vicente de Fora
38.71461291666666 mediterranean 3 São Vicente de Fora
38.71461291666666 european 2 São Vicente de Fora
38.71461291666666 european 1 São Vicente de Fora
38.71461291666666 european 4 São Vicente de Fora
38.71461291666666 european 3 São Vicente de Fora
38.71461291666666 chicken 2 São Vicente de Fora
38.71461291666666 chicken 1 São Vicente de Fora
38.71461291666666 chicken 4 São Vicente de Fora
38.71461291666666 chicken 3 São Vicente de Fora
38.71461291666666 bar 2 São Vicente de Fora
38.71461291666666 bar 1 São Vicente de Fora
38.71461291666666 bar 4 São Vicente de Fora
38.71461291666666 bar 3 São Vicente de Fora
38.71461291666666 seafood 2 São Vicente de Fora
38.71461291666666 seafood 1 São Vicente de Fora
38.71461291666666 seafood 4 São Vi

...

38.709874694444444 pizza 3 Mártires
38.709874694444444 japanese 2 Mártires
38.709874694444444 japanese 1 Mártires
38.709874694444444 japanese 4 Mártires
38.709874694444444 japanese 3 Mártires
38.709874694444444 pasta 2 Mártires
38.709874694444444 pasta 1 Mártires
38.709874694444444 pasta 4 Mártires
38.709874694444444 pasta 3 Mártires
38.709874694444444 mexican 2 Mártires
38.709874694444444 mexican 1 Mártires
38.709874694444444 mexican 4 Mártires
38.709874694444444 mexican 3 Mártires
38.709874694444444 africa_me 2 Mártires
38.709874694444444 africa_me 1 Mártires
38.709874694444444 africa_me 4 Mártires
38.709874694444444 africa_me 3 Mártires
38.709874694444444 burger 2 Mártires
38.709874694444444 burger 1 Mártires
38.709874694444444 burger 4 Mártires
38.709874694444444 burger 3 Mártires
38.709874694444444 italian 2 Mártires
38.709874694444444 italian 1 Mártires
38.709874694444444 italian 4 Mártires
38.709874694444444 italian 3 Mártires
38.709874694444444 asian 2 Mártires
38.7098746944444

38.711333759999995 indian_nepalese 4 São Nicolau
38.711333759999995 indian_nepalese 3 São Nicolau
38.711333759999995 portuguese 2 São Nicolau
38.711333759999995 portuguese 1 São Nicolau
38.711333759999995 portuguese 4 São Nicolau
38.711333759999995 portuguese 3 São Nicolau
38.711333759999995 pizza 2 São Nicolau
38.711333759999995 pizza 1 São Nicolau
38.711333759999995 pizza 4 São Nicolau
38.711333759999995 pizza 3 São Nicolau
38.711333759999995 japanese 2 São Nicolau
38.711333759999995 japanese 1 São Nicolau
38.711333759999995 japanese 4 São Nicolau
38.711333759999995 japanese 3 São Nicolau
38.711333759999995 pasta 2 São Nicolau
38.711333759999995 pasta 1 São Nicolau
38.711333759999995 pasta 4 São Nicolau
38.711333759999995 pasta 3 São Nicolau
38.711333759999995 mexican 2 São Nicolau
38.711333759999995 mexican 1 São Nicolau
38.711333759999995 mexican 4 São Nicolau
38.711333759999995 mexican 3 São Nicolau
38.711333759999995 africa_me 2 São Nicolau
38.711333759999995 africa_me 1 São Nico

38.71284574 bar 2 Santiago
38.71284574 bar 1 Santiago
38.71284574 bar 4 Santiago
38.71284574 bar 3 Santiago
38.71284574 seafood 2 Santiago
38.71284574 seafood 1 Santiago
38.71284574 seafood 4 Santiago
38.71284574 seafood 3 Santiago
38.71284574 south_america 2 Santiago
38.71284574 south_america 1 Santiago
38.71284574 south_america 4 Santiago
38.71284574 south_america 3 Santiago
38.71284574 veggie_healthy 2 Santiago
38.71284574 veggie_healthy 1 Santiago
38.71284574 veggie_healthy 4 Santiago
38.71284574 veggie_healthy 3 Santiago
38.71284574 bistro 2 Santiago
38.71284574 bistro 1 Santiago
38.71284574 bistro 4 Santiago
38.71284574 bistro 3 Santiago
38.71284574 grill 2 Santiago
38.71284574 grill 1 Santiago
38.71284574 grill 4 Santiago
38.71284574 grill 3 Santiago
38.71284574 indian_nepalese 2 Santiago
38.71284574 indian_nepalese 1 Santiago
38.71284574 indian_nepalese 4 Santiago
38.71284574 indian_nepalese 3 Santiago
38.71284574 portuguese 2 Santiago
38.71284574 portuguese 1 Santiago
38.71284

38.7354768 veggie_healthy 3 Alvalade
38.7354768 bistro 2 Alvalade
38.7354768 bistro 1 Alvalade
38.7354768 bistro 4 Alvalade
38.7354768 bistro 3 Alvalade
38.7354768 grill 2 Alvalade
38.7354768 grill 1 Alvalade
38.7354768 grill 4 Alvalade
38.7354768 grill 3 Alvalade
38.7354768 indian_nepalese 2 Alvalade
38.7354768 indian_nepalese 1 Alvalade
38.7354768 indian_nepalese 4 Alvalade
38.7354768 indian_nepalese 3 Alvalade
38.7354768 portuguese 2 Alvalade
38.7354768 portuguese 1 Alvalade
38.7354768 portuguese 4 Alvalade
38.7354768 portuguese 3 Alvalade
38.7354768 pizza 2 Alvalade
38.7354768 pizza 1 Alvalade
38.7354768 pizza 4 Alvalade
38.7354768 pizza 3 Alvalade
38.7354768 japanese 2 Alvalade
38.7354768 japanese 1 Alvalade
38.7354768 japanese 4 Alvalade
38.7354768 japanese 3 Alvalade
38.7354768 pasta 2 Alvalade
38.7354768 pasta 1 Alvalade
38.7354768 pasta 4 Alvalade
38.7354768 pasta 3 Alvalade
38.7354768 mexican 2 Alvalade
38.7354768 mexican 1 Alvalade
38.7354768 mexican 4 Alvalade
38.7354768 me

38.7124757 mexican 3 Almargem do Bispo
38.7124757 africa_me 2 Almargem do Bispo
38.7124757 africa_me 1 Almargem do Bispo
38.7124757 africa_me 4 Almargem do Bispo
38.7124757 africa_me 3 Almargem do Bispo
38.7124757 burger 2 Almargem do Bispo
38.7124757 burger 1 Almargem do Bispo
38.7124757 burger 4 Almargem do Bispo
38.7124757 burger 3 Almargem do Bispo
38.7124757 italian 2 Almargem do Bispo
38.7124757 italian 1 Almargem do Bispo
38.7124757 italian 4 Almargem do Bispo
38.7124757 italian 3 Almargem do Bispo
38.7124757 asian 2 Almargem do Bispo
38.7124757 asian 1 Almargem do Bispo
38.7124757 asian 4 Almargem do Bispo
38.7124757 asian 3 Almargem do Bispo
38.7124757 pastry 2 Almargem do Bispo
38.7124757 pastry 1 Almargem do Bispo
38.7124757 pastry 4 Almargem do Bispo
38.7124757 pastry 3 Almargem do Bispo
38.7124757 fast_food 2 Almargem do Bispo
38.7124757 fast_food 1 Almargem do Bispo
38.7124757 fast_food 4 Almargem do Bispo
38.7124757 fast_food 3 Almargem do Bispo
38.7124757 fado 2 Almarge

SyntaxError: unexpected EOF while parsing (1324140925.py, line 1)

In [328]:
X, y = restaurant_data()
all_scenarios = []
address = None
type_of_food = X['type'].unique()
neighborhood = X['neighborhood'].unique()
price = X['price'].unique()
pipeline = get_fitted_pipe(X)
for n in neighborhood:  
    latitude1, longitude1 = get_imputed_coords(n, X)
    for t in type_of_food:
        for p in price:
            X_user = transform_input(t, p, n, latitude1, longitude1)
            X_transformed = pipeline.transform(X_user)
            loaded_model = joblib.load('knn_model.joblib')
            y_predicted_prob = loaded_model.predict_proba(X_transformed)
            all_scenarios.append({'type_of_food':t, 'neighborhood':n, 'price':p, 'outputs': y_predicted_prob[0][0]})
df = pd.DataFrame(all_scenarios)   
df = df.sort_values(by=['outputs'], ascending=False)
df.to_csv('data_probabilities.csv')

In [329]:
df = pd.DataFrame(all_scenarios)

In [339]:
df = df.sort_values(by=['outputs'], ascending=False)

In [341]:
df.outputs.value_counts()

0.8    1168
0.6    1091
0.4     656
1.0     548
0.2     167
0.0      18
Name: outputs, dtype: int64

In [342]:
df.to_csv('data_probabilities.csv')

In [376]:
df

,type_of_food,neighborhood,price,outputs
0,italian,Santiago,2,1.0
1,fado,Santiago,3,1.0
2,italian,Sacramento,3,1.0
3,asian,Sacramento,2,1.0
4,asian,Sacramento,3,1.0
...,...,...,...,...
3643,pastry,Prazeres,1,0.0
3644,chicken,Santa Catarina,2,0.0
3645,cafe,Prazeres,1,0.0
3646,chicken,Prazeres,1,0.0


In [375]:
df = pd.read_csv('../../notebooks/3_Prediction/data_probabilities.csv')
df = df.drop(columns = 'Unnamed: 0')
rows = []
type_of_food =['brunch', 'chicken']
price = [2, 1]
neighborhood = ['Lapa', 'Graça']
for t in type_of_food:
    for p in price:
        for n in neighborhood: 
            prediction_row = df.loc[(df['type_of_food'] == type_of_food[t]) & (df['neighborhood'] == neighborhood[n]) & (df['price'] == price[p])]
            rows.append(prediction_row)
#df = pd.DataFrame(rows)
#df = df.sort_values(by=['outputs'], ascending=False)

TypeError: list indices must be integers or slices, not str

In [378]:
df = pd.read_csv('../../notebooks/3_Prediction/data_probabilities.csv')
df = df.drop(columns = 'Unnamed: 0')
rows = []
type_of_food =['brunch', 'chicken']
price = [2, 1]
neighborhood = ['Lapa', 'Graça']

prediction_row = df.loc[(df['type_of_food'].isin(type_of_food)) & (df['neighborhood'].isin(neighborhood)) & (df['price'].isin(price))]
prediction_row

,type_of_food,neighborhood,price,outputs
398,chicken,Graça,2,1.0
399,chicken,Graça,1,1.0
2434,brunch,Graça,1,0.6
2657,brunch,Graça,2,0.6
3443,chicken,Lapa,2,0.4
3539,brunch,Lapa,1,0.2
3540,brunch,Lapa,2,0.2
3563,chicken,Lapa,1,0.2


In [362]:
neighborhood[1]

'Graça'

In [374]:
df.loc[(df['type_of_food'] == 'brunch') & (df['neighborhood'] == 'Lapa') & (df['price'] == 3)]

,type_of_food,neighborhood,price,outputs
3461,brunch,Lapa,3,0.4
